# <u><b> League of Legends winner by 10 minutes</b></u>

In this notebook we try to make a model that can predict the winner in a hugh elo (high diamond) game of league of legends using classification.

The paramaters that we are using are as follows:

* gameId - Unique RIOT ID of the game. Can be used with the Riot Games API
* blueWins - The target column. 1 if the blue team has won, 0 otherwise
* blueWardsPlaced - Number of warding totems placed by the blue team on the map
* blueWardsDestroyed - Number of enemy warding totems the blue team has destroyed
* blueFirstBlood - First kill of the game. 1 if the blue team did the first kill, 0 otherwise
* blueKills - Number of enemies killed by the blue team
* blueDeaths - Number of deaths (blue team)
* blueAssists - Number of kill assists (blue team)
* blueEliteMonsters - Number of elite monsters killed by the blue team (Dragons and Heralds)
* blueDragons - Number of dragons killed by the blue team
* blueHeralds - Number of heralds killed by the blue team
* blueTowersDestroyed - Number of towers destroyed by the blue team
* blueTotalGold - Blue team total gold
* blueAvgLevel - Blue team average champion level
* blueTotalExperience - Blue team total experience
* blueTotalMinionsKilled - Blue team total minions killed (CS)
* blueTotalJungleMinionsKilled - Blue team total jungle monsters killed
* blueGoldDiff - Blue team gold difference compared to the enemy team
* blueExperienceDiff - Blue team experience difference compared to the enemy team
* blueCSPerMin - Blue team CS (minions) per minute
* blueGoldPerMin - Blue team gold per minute
* redWardsPlaced - Number of warding totems placed by the red team on the map
* redWardsDestroyed - Number of enemy warding totems the red team has destroyed
* redFirstBlood - First kill of the game. 1 if the red team did the first kill, 0 otherwise
* redKills - Number of enemies killed by the red team
* redDeaths - Number of deaths (red team)
* redAssists - Number of kill assists (red team)
* redEliteMonsters - Number of elite monsters killed by the red team (Dragons and Heralds)
* redDragons - Number of dragons killed by the red team
* redHeralds - Number of heralds killed by the red team
* redTowersDestroyed - Number of towers destroyed by the red team
* redTotalGold - Red team total gold
* redAvgLevel - Red team average champion level
* redTotalExperience - Red team total experience
* redTotalMinionsKilled - Red team total minions killed (CS)
* redTotalJungleMinionsKilled - Red team total jungle monsters killed
* redGoldDiff - Red team gold difference compared to the enemy team
* redExperienceDiff - Red team experience difference compared to the enemy team
* redCSPerMin - Red team CS (minions) per minute
* redGoldPerMin - Red team gold per minute

In [1]:
#imports
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, plot_confusion_matrix, roc_curve, plot_roc_curve
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
import xgboost as xgb


ModuleNotFoundError: No module named 'xgboost'

## <u><b> Part 1 </b></u>

### Adjusting the data for program


#### Loading the data

In [ ]:
rnd = 42
np.random.seed(rnd)
df = pd.read_csv(r'data/high_diamond_ranked_10min.csv')
df.head()


In [ ]:
df.info()

In [ ]:
for i in df.columns:
    print("* "+str(i)+" - ")

#### Looking for missing values

In [ ]:
df.isna().sum()

## <u><b> Part 2 </b></u>

### Cleaning and scaling the data

#### Removing known useless data 
We know that the game ID has 0 impact on who wins the game so we drop it

In [ ]:
df.drop(columns=["gameId"],inplace=True)
data = df.copy()

#### Visualization of the correlations between the columns

In [ ]:
matrix = data.corr()
plt.figure(figsize=(16,12))
sns.heatmap(matrix)
plt.show()

#### Remove correlated features
Highly correlated data can slow our model while not offering any precision, especialy in games like League of Legends where once you have more kills you (usualy) have more gold that lets you get even more kills and objectives. Some features like redDeaths and blueKills are perfectly correlated so we don't need one of them at all.

In [ ]:
def correlations(D,mainCorr=None, limit=0.95):
    #dataframe of boolean values of D dataFrame correlations, true iff |value|>=0.95
    highCorrColl = abs(D.corr())>=limit
    colls = []
    remove = []
    for coll in highCorrColl.columns:
        #we might delete a column we didn't visit
        if coll in highCorrColl.columns:
            #if any of the values is true and its not the same index as the column (since at (x,x) we always have a value of 1 so it will always be true)
            if highCorrColl[coll][highCorrColl[coll].index!=coll].any():
                #mainCorr is a series of correlation values of the main feature and the rest of the columns 
                if mainCorr is not None:
                    #get all the elements that have a high correlation with coll (including coll, (coll,coll) is always true) 
                    correl = highCorrColl[coll]
                    correl = correl[correl==True]

                    #get the representation of correl in the correlation series mainCorr
                    correlVals = mainCorr[correl.index]
                    
                    #indexs to remove, we want to keep the ones with the highest correlations to mainCorr/"blueWins"
                    removeIDX = correlVals[abs(correlVals).idxmax() != correlVals.index].index
                    highCorrColl = highCorrColl.drop(columns = removeIDX).drop(index = removeIDX)
                    remove.extend(removeIDX)
                else:
                    #if mainCorr was not passed, drop coll
                    highCorrColl = highCorrColl.drop(columns = coll).drop(index = coll)
                    remove.append(coll)
    return D.drop(columns=remove)

#custom class for pipeline
class FilterCorrelations(BaseEstimator, TransformerMixin):
    def __init__(self,mainCorr=None,limit=0.95):
        self.mainCorr=mainCorr
        self.limit = limit
    def fit(self, Data):
        return self
    def transform(self,Data):
        return correlations(Data,mainCorr=self.mainCorr, limit=self.limit)

#### Scale data
We will use sklearn's standard scaler since our data has many outliers since sometimes player "throw" a game resulting in an unexpected win or loss

In [ ]:
#custom class for pipeline
class Scale(BaseEstimator, TransformerMixin):
    def fit(self, Data):
        return self
    def transform(self,Data):
        tempDF = Data.copy()
        Data = StandardScaler().fit_transform(Data)
        return pd.DataFrame(Data,columns=tempDF.columns,index=tempDF.index)

#### Find labels wihout correlation with winning

In [ ]:
(abs(matrix["blueWins"])<=0).any()

## <u><b> Part 3 </b></u>

### Splitting data to train/test and making models

In [ ]:
exp = df["blueWins"]
data = df.drop(columns=["blueWins"])

In [ ]:
trainD, testD, trainE, testE = train_test_split(data, exp,random_state=rnd)
pipe = Pipeline([
    ("handle_correlations",FilterCorrelations(mainCorr=df.corr()["blueWins"],limit=0.95)),
    ('scaler',Scale())
])

In [ ]:
trainD = pipe.fit_transform(trainD)
testD = pipe.transform(testD)

#### Finding lables with high importance
Since we have so many labels, we could try and remove some so the program could run faster without loss of accuracy

In [ ]:
forest = ExtraTreesClassifier(n_estimators=250,random_state=rnd)
forest.fit(trainD,trainE)

In [ ]:
importanceDF = pd.DataFrame({"label":trainD.columns, "importances":forest.feature_importances_})
importanceDF.sort_values(by="importances",ascending=False,inplace=True)
importanceDF.plot(x="label",y="importances",kind="barh",figsize=(12,12))

In [ ]:
trainD.drop(columns=trainD.columns.difference(importanceDF.head(8)["label"]),inplace=True)
testD.drop(columns=testD.columns.difference(importanceDF.head(8)["label"]),inplace=True)
importanceDF.head(8)["label"]

In [ ]:
trainD.head()

### KNN model


#### K=5
This will be our baseline model, we will try to get better results

In [ ]:
knn5_clf = KNeighborsClassifier(n_neighbors=5)
knn5_clf.fit(trainD,trainE)
knn5_pred = knn5_clf.predict(testD)

In [ ]:
knn5_accuracy = accuracy_score(testE,knn5_pred)
print("Accuracy:",knn5_accuracy)
print(classification_report(testE,knn5_pred))

#### Paramater tuning the for KNN model

In [ ]:
knn_clf = KNeighborsClassifier(n_neighbors=5)
#Parameters we want to use in order to find the best KNN model
param_grid=[
    {'n_neighbors':np.arange(1,31),'weights':["uniform","distance"]}
]
#change n_jobs for faster/slower performance

gridSearch = RandomizedSearchCV(knn_clf, param_grid, n_iter=30, scoring='accuracy', return_train_score=True,verbose=4,n_jobs=-1,random_state=rnd)

In [ ]:
gridSearch.fit(trainD,trainE)

In [ ]:
uniforms = []
distances = []
for i in range(len(gridSearch.cv_results_['params'])):
    res = gridSearch.cv_results_['params'][i]
    #make a dictionary with {n_neighbors,weights,score} and append it in the right list
    res['score'] = gridSearch.cv_results_["mean_test_score"][i]
    if res["weights"]=="uniform": 
        uniforms.append(res)
    else:
        distances.append(res)
models = np.asarray([uniforms,distances])

Visualizing the performance of the models

In [ ]:
fig = plt.figure(figsize=(16,12))
ax = fig.add_axes([0,0,1,1])
df1 = pd.DataFrame(models[0]).sort_values(by="n_neighbors")
df2 = pd.DataFrame(models[1]).sort_values(by="n_neighbors")
ax.plot(df1["n_neighbors"],df1["score"],"ro-")
ax.plot(df2["n_neighbors"],df2["score"],"bo-")
#place a green dot at the best classifier
ax.plot(gridSearch.best_estimator_.n_neighbors,gridSearch.best_score_,"go")
ax.set_ylabel("mean test score")
ax.set_xlabel("n_neighbors")
ax.legend(labels=["uniform","distance","best classifier score"])
ax.set_xticks([i for i in range(5*7+1)],minor=True)
ax.set_xticks([5*i for i in range(8)],minor=False)

plt.show()

#### Fitting KNN model

In [ ]:
knn_clf = gridSearch.best_estimator_
knn_clf.fit(trainD,trainE)
knn_pred = knn_clf.predict(testD)
knn_clf

In [ ]:
knn_accuracy = accuracy_score(testE,knn_pred)
print("Accuracy:",knn_accuracy)
print(classification_report(testE,knn_pred))

### SVC model

#### Paramater tuning for the SVC model

In [ ]:
svc_clf = SVC(random_state=rnd)
#Parameters we want to use in order to find the best KNN model
param_grid=[
    {'C':np.linspace(0.9,1.1,30),"gamma":["auto"]},
    {'C':np.linspace(0.9,1.1,30),"gamma":["scale"]}
]
#change n_jobs for faster/slower performance
gridSearch = RandomizedSearchCV(svc_clf, param_grid, n_iter=20, scoring='accuracy', return_train_score=True,verbose=4,n_jobs=-1,random_state=rnd)


In [ ]:
gridSearch.fit(trainD,trainE)

In [ ]:
gridSearch.cv_results_['params'][0]

In [ ]:
scale = []
auto = []
for i in range(len(gridSearch.cv_results_['params'])):
    res = gridSearch.cv_results_['params'][i]
    #make a dictionary with {n_neighbors,weights,score} and append it in the right list
    res['score'] = gridSearch.cv_results_["mean_test_score"][i]
    if res["gamma"]=="scale":
        scale.append(res)
    else:
        auto.append(res)
models = np.asarray([scale,auto])

#### Visualizing the performance of the models

In [ ]:
fig = plt.figure(figsize=(16,12))
ax = fig.add_axes([0,0,1,1])
df1 = pd.DataFrame(models[0]).sort_values(by="C")
df2 = pd.DataFrame(models[1]).sort_values(by="C")
ax.plot(df1["C"],df1["score"],"ro-")
ax.plot(df2["C"],df2["score"],"bo-")
#place a green dot at the best classifier
ax.plot(gridSearch.best_estimator_.C,gridSearch.best_score_,"go")
ax.set_ylabel("mean test score")
ax.set_xlabel("C")
ax.legend(labels=["gamma=scale","gamma=auto","best classifier score"])
plt.show()

Fitting SVC model

In [ ]:
svc_clf = gridSearch.best_estimator_
svc_clf.fit(trainD,trainE)
svc_pred = svc_clf.predict(testD)
svc_clf

In [ ]:
svc_accuracy = accuracy_score(testE,svc_pred)
print("Accuracy:",svc_accuracy)
print(classification_report(testE,svc_pred))

## <u><b> Part 4 </b></u>

### Results

#### Confusion matrix

In [ ]:
fig,axes = plt.subplots(ncols=3,figsize=(20,8),sharex=True)
cms = [None,None,None]
cms[0] = plot_confusion_matrix(knn5_clf,testD,testE,ax=axes[0],display_labels=["Red","Blue"],values_format="d")
cms[1] = plot_confusion_matrix(knn_clf,testD,testE,ax=axes[1],display_labels=["Red","Blue"],values_format="d")
cms[2] = plot_confusion_matrix(svc_clf,testD,testE,ax=axes[2],display_labels=["Red","Blue"],values_format="d")
vals = [["True Neg","False Pos"],["False Neg","True Pos"]]
for cm in cms:
    text = [[cm.text_[i][j].get_text() for j in range(len(cm.text_[i]))] for i in range(len(cm.text_))]
    for i in range(len(text)):
        for t in range(len(text[i])):
            txt = vals[i][t] + "\n" + text[i][t] + "\n" + '{0:.3f}'.format(100*(int(text[i][t])/len(testE)))+"%"
            cm.text_[i][t].set_text(txt)
        
axes[0].title.set_text("KNN\nn = 5")
axes[1].title.set_text("KNN\nn ="+str(knn_clf.n_neighbors))
axes[2].title.set_text("SVC\nC ="+str('{0:.2f}'.format(svc_clf.C)))



#### ROC curve

In [ ]:
fig = plt.figure(figsize=(16,12))
ax = fig.add_axes([0,0,1,1])
falsePosRate, truePosRate, _ = roc_curve(testE,np.random.choice([0,1],size=len(testE)))
plot_roc_curve(knn5_clf,testD,testE,ax=ax,name="KNN, n = 5")
plot_roc_curve(knn_clf,testD,testE,ax=ax,name="KNN, n ="+str(knn_clf.n_neighbors))
plot_roc_curve(svc_clf,testD,testE,ax=ax,name="SVC, C ="+str('{0:.2f}'.format(svc_clf.C)))
ax.plot(falsePosRate, truePosRate, linestyle='--')

#### Accuracy rate


In [ ]:
print("Accuracy rate of KNN model (n = 5):",accuracy_score(testE,knn5_pred))
print("Accuracy rate of KNN model (n = " + str(knn_clf.n_neighbors) + "):",accuracy_score(testE,knn_pred))
print("Accuracy rate of SVC model (C = " + str('{0:.2f}'.format(svc_clf.C)) + "):",accuracy_score(testE,svc_pred))

## <u><b> Conclusions </b></u>

70%-73% seems like a low accuracy but considering the dataset is based on the first 10 minutes of a game, having anything over 65% is incredibly good!

This data shows us that although a large amount of the community think the game is over because of a single mistake at the start of the game, this is not actually the case.

In addition, the dataset doesn't list the champions on each team, some chamions are really good at the early part of the game like Renekton and Lee sin, but tend to fall off in the mid-late game while chamions like Veigar and Nasus tend to have a hard time at the early parts of the game, but can solo carry a game if kept unchecked.

Having only 70%-73% accuracy score is actually a very good sign! Since it means that players can't know who would win this early, meaning that their performance in the game won't change if they find out they have a 90% chance to lose or win, this can make it very frustrating, even for the winning team because the losing team could choose to surrender at minute 15 and not allow the winning team enjoy their win properly. 

Since we only have data from the 10 minute mark, we can only get a 70%-73% accuracy score, but if we had data from minute 15 or 20, I would expect an accuracy rate closer to 80%-85%.

## <u><b> Part 5 </b></u>

### Improvements

Now we will try to imorove our model, lets see if we can reach a better accuracy and at the very least, reduce the amount of features we need to get an accurate result

In [ ]:
from sklearn.decomposition import PCA
import time
from sklearn.preprocessing import Normalizer

In [ ]:
df.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = ["blueWins"]), df["blueWins"], random_state=rnd)


In [ ]:
X_train_new = X_train.drop(columns=X_train.columns.difference(importanceDF.head(8)["label"]))
X_test_new = X_test.drop(columns=X_train.columns.difference(importanceDF.head(8)["label"]))
importanceDF.head(8)["label"]

In [ ]:
scale = Normalizer()
X_train_new = scale.fit_transform(X_train_new)
X_test_new = scale.transform(X_test_new)

In [ ]:
scale = StandardScaler()

#X_train_new = scale.fit_transform(X_train_new)
#X_test_new = scale.transform(X_test_new)

#### PCA
Let's first start by reducing our model's complexity using PCA 

In [ ]:
pca = PCA(random_state=rnd)
pca.fit(X_train_new)
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
pca.fit(X_train_new)
exp_var_cumul2 = np.cumsum(pca.explained_variance_ratio_)

In [ ]:
fig, ax = plt.subplots()
ax.plot(range(1,exp_var_cumul.shape[0] + 1),exp_var_cumul)
ax.fill_between(range(1,exp_var_cumul.shape[0] + 1),exp_var_cumul,0)



In [ ]:
dim = len(exp_var_cumul[exp_var_cumul<0.95])
exp_var_cumul[dim], dim

In [ ]:
pipe = Pipeline([
    ("normalizer",Normalizer()),
    ("pca",PCA(n_components=dim,random_state=rnd))
],verbose=True)

In [ ]:
X_train_new = pipe.fit_transform(X_train_new)
X_test_new = pipe.transform(X_test_new)

In [ ]:
X_train_new

In [ ]:
dtrain = xgb.DMatrix(X_train_new, label=y_train)
dtest = xgb.DMatrix(X_test_new, label=y_test)

#### paramater tuning


def param_search(param_grid,train,test,train_labels,test_labels,randomSeed=rnd,nfolds=5,eval_method="auc",num_class=2,num_boost_round=10000,early_stopping_rounds=40,verbose=True):
    done = 0
    amount = len(param_grid)
    params = {
        'random_state':randomSeed,
        'eval_metric':eval_method,
    }
    start = time.time()
    timer = time.time()
    best_eval = -float("inf")
    best_eval_train = 0
    best_params = {}
    best_iter = 0
    best_out = ''
    for p in param_grid:
        out = ''
        for key in p:
            params[key] = p[key]
            out += key + '=' + str(p[key])+', '
        out = out[:-2]
        
        results = xgb.cv(params,train,num_boost_round=num_boost_round,nfold=nfolds,metrics={eval_method},early_stopping_rounds=early_stopping_rounds,seed=randomSeed)
        
        done +=1 
        #get mean score of cv of the training and testing, and get the best iteration
        mean_eval_test = results['test-'+eval_method+'-mean'].min()
        mean_eval_train = results['test-'+eval_method+'-mean'].min()
        iteration = results['test-'+eval_method+'-mean'].argmin()
        
        #update best parameters
        if mean_eval_test > best_eval:
            best_eval = mean_eval_test
            best_eval_train = mean_eval_train
            best_out = ''
            best_iter = iteration
            for key in p:
                best_out += key + '=' + str(p[key])+', '
                best_params[key] = p[key]
            best_out = best_out[:-2]

        #show results of cv
        if verbose:
            print("Parameters: "+out)
            print("Mean train "+eval_method+": "+str(mean_eval_train) + " Mean test "+eval_method+": "+str(mean_eval_test) + " Iteration: "+str(iteration))
            print(str(done)+"/"+str(amount))
            
            if (time.time()-timer) > 300:
                print(str((time.time()-start)/60) +"m elapsed. Finished "+str(100*done/amount)+"%" )
                timer = time.time()
            print("")
    stop = time.time()
    if verbose:
        print("----------------------------------------")
    print(f"Tuning time: {(time.time()-start)}s")
    print("Best parameters: "+best_out+":")
    print("Best mean train "+eval_method+": "+str(best_eval_train) + " Best mean test "+eval_method+": "+str(best_eval) + " Best iteration: "+str(iteration))
    
    #show accuracy score
    bst = xgb.train(dict({'random_state':randomSeed,'eval_metric':eval_method,'num_class':num_class},**best_params), train, best_iter+1)
    test_preds = bst.predict(test)
    train_preds = bst.predict(train)

    print("Test accuracy:",accuracy_score(test_labels,test_preds),"Train accuracy:",accuracy_score(train_labels,train_preds))
    return best_params,best_iter+1

In [ ]:
def param_search(param_grid,train,test,train_labels,test_labels,num_class=2,randomSeed=rnd,nfolds=5,eval_method="auc",higher_better=True,num_boost_round=10000,early_stopping_rounds=40,verbose=True):
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(20,10))
    done = 0
    amount = len(param_grid)
    params = {
        'random_state':randomSeed,
        'eval_metric':eval_method,
    }
    start = time.time()
    timer = time.time()
    if higher_better: 
        best_eval = -float("inf")
    else:
        best_eval = float("inf")
    best_eval_train = 0
    best_params = {}
    best_iter = 0
    best_out = ''
    with tqdm(total=len(param_grid)) as pbar:
        for p in param_grid:
            out = ''
            for key in p:
                params[key] = p[key]
                out += key + '=' + str(p[key])+', '
            out = out[:-2]
            
            results = xgb.cv(params,train,num_boost_round=num_boost_round,nfold=nfolds,metrics={eval_method},early_stopping_rounds=early_stopping_rounds,seed=randomSeed)
            
            ax1.plot(results.index,results['test-'+eval_method+'-mean'],label=out[:25])
            ax2.plot(results.index,results['train-'+eval_method+'-mean'],label=out[:25])
            ax1.set_title("test")
            ax2.set_title("train")
            ax1.legend()
            ax2.legend()
            
            done +=1 
            #get mean score of cv of the training and testing, and get the best iteration
            if higher_better==False:
                mean_eval_test = results['test-'+eval_method+'-mean'].min()
                mean_eval_train = results['test-'+eval_method+'-mean'].min()
                iteration = results['test-'+eval_method+'-mean'].argmin()
            else: 
                mean_eval_test = results['test-'+eval_method+'-mean'].max()
                mean_eval_train = results['test-'+eval_method+'-mean'].max()
                iteration = results['test-'+eval_method+'-mean'].argmax()
            
            #update best parameters
            if (mean_eval_test < best_eval and higher_better==False) or (mean_eval_test > best_eval and higher_better):
                best_eval = mean_eval_test
                best_eval_train = mean_eval_train
                best_out = ''
                best_iter = iteration
                for key in p:
                    best_out += key + '=' + str(p[key])+', '
                    best_params[key] = p[key]
                best_out = best_out[:-2]

            #show results of cv
            if verbose:
                print("Parameters: "+out)
                print("Mean train "+eval_method+": "+str(mean_eval_train) + " Mean test "+eval_method+": "+str(mean_eval_test) + " Iteration: "+str(iteration))
                print(str(done)+"/"+str(amount))
                
                if (time.time()-timer) > 300:
                    print(str((time.time()-start)/60) +"m elapsed. Finished "+str(100*done/amount)+"%" )
                    timer = time.time()
                print("")
            pbar.update(1)
    stop = time.time()
    if verbose:
        print("----------------------------------------")
    print(f"Tuning time: {(time.time()-start)}s")
    print("Best parameters: "+best_out+":")
    print("Best mean train "+eval_method+": "+str(best_eval_train) + " Best mean test "+eval_method+": "+str(best_eval) + " Best iteration: "+str(iteration))
    
    #show accuracy score
    bst = xgb.train(dict({'random_state':randomSeed,'eval_metric':eval_method,'num_class':num_class},**best_params), train, best_iter+1)
    test_preds = bst.predict(test)
    train_preds = bst.predict(train)

    print("Test accuracy:",accuracy_score(test_labels,test_preds),"Train accuracy:",accuracy_score(train_labels,train_preds))
    return best_params,best_iter+1

In [ ]:
params = {
    'tree_method':'hist',
    'n_jobs':-1
}

In [ ]:
grid = [
    dict({'learning_rate':lr},**params) 
    for lr in [0.001,0.005,0.01,0.05,0.1,0.2,0.25,0.3,0.35,0.4,0.5,0.7]
]

In [ ]:
params,_ = param_search(grid,dtrain,dtest,y_train,y_test)

In [ ]:
grid = [
    dict({'max_depth':i,'min_child_weight':j},**params) 
    for i in [1,3,6,9,12]
    for j in [1,3,5,8,10]
]

In [ ]:
params,_ = param_search(grid,dtrain,dtest,y_train,y_test)

In [ ]:
grid = [
    dict({'gamma':g},**params) 
    for g in [0,1,2,3,4,5]
]

In [ ]:
params,_ = param_search(grid,dtrain,dtest,y_train,y_test)

In [ ]:
grid = [
    dict({'subsample':i,'colsample_bytree':j},**params) 
    for i in [0.7,0.8,0.9,1]
    for j in [0.7,0.8,0.9,1]
]

In [ ]:
params,_ = param_search(grid,dtrain,dtest,y_train,y_test)

In [ ]:
grid = [
    dict({'reg_lambda':i},**params) 
    for i in [1,5,10,15,20,25]
]

In [ ]:
params,estimators = param_search(grid,dtrain,dtest,y_train,y_test)

### Final model

In [ ]:
model = xgb.XGBClassifier(n_estimators = estimators, random_state=rnd,**params,eval_metric="mae")


In [ ]:
model.fit(X_train_new,y_train)

In [ ]:
pred_test = model.predict(X_test_new)
pred_train = model.predict(X_train_new)

#### Results

In [ ]:
def plot_model_results(model,X,y,title=""):
   
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.suptitle(title)
    cm = plot_confusion_matrix(model,X,y,display_labels=["Red","Blue"],values_format="d",ax=ax1)
    fig.set_size_inches(20,10)
    num = cm.text_[0][0].get_text()
    txt = "True Negative"+ "\n" + num + "\n" + '{0:.3f}'.format(100*(int(num)/len(y)))+"%"
    cm.text_[0][0].set_text(txt)
    
    num = cm.text_[1][0].get_text()
    txt = "False Negative"+ "\n" + num + "\n" + '{0:.3f}'.format(100*(int(num)/len(y)))+"%"
    cm.text_[1][0].set_text(txt)

    num = cm.text_[0][1].get_text()
    txt = "False Positive"+ "\n" + num + "\n" + '{0:.3f}'.format(100*(int(num)/len(y)))+"%"
    cm.text_[0][1].set_text(txt)

    num = cm.text_[1][1].get_text()
    txt = "True Positive"+ "\n" + num + "\n" + '{0:.3f}'.format(100*(int(num)/len(y)))+"%"
    cm.text_[1][1].set_text(txt)
    
    falsePosRate, truePosRate, _ = roc_curve(y,np.random.choice([0,1],size=len(y)))
    plot_roc_curve(model,X,y,ax=ax2)
    ax2.plot(falsePosRate, truePosRate, linestyle='--')

In [ ]:
print("Train accuracy:",accuracy_score(y_train,pred_train))
print(classification_report(y_train,pred_train))

In [ ]:

plot_model_results(model,X_train_new,y_train,title="train")

In [ ]:
model_accuracy = accuracy_score(y_test,pred_test)
print("Test accuracy:",model_accuracy)
print(classification_report(y_test,pred_test))

In [ ]:

plot_model_results(model,X_test_new,y_test,title="test")

In [ ]:
fig,axes = plt.subplots(ncols=4,figsize=(28,8),sharex=True)
cms = [None,None,None,None]
cms[0] = plot_confusion_matrix(knn5_clf,testD,testE,ax=axes[0],display_labels=["Red","Blue"],values_format="d")
cms[1] = plot_confusion_matrix(knn_clf,testD,testE,ax=axes[1],display_labels=["Red","Blue"],values_format="d")
cms[2] = plot_confusion_matrix(svc_clf,testD,testE,ax=axes[2],display_labels=["Red","Blue"],values_format="d")
cms[3] = plot_confusion_matrix(model,X_test_new,y_test,ax=axes[3],display_labels=["Red","Blue"],values_format="d")
vals = [["True Neg","False Pos"],["False Neg","True Pos"]]
for cm in cms:
    text = [[cm.text_[i][j].get_text() for j in range(len(cm.text_[i]))] for i in range(len(cm.text_))]
    for i in range(len(text)):
        for t in range(len(text[i])):
            txt = vals[i][t] + "\n" + text[i][t] + "\n" + '{0:.3f}'.format(100*(int(text[i][t])/len(testE)))+"%"
            cm.text_[i][t].set_text(txt)
        
axes[0].title.set_text("KNN\nn = 5\n"+str(knn5_accuracy))
axes[1].title.set_text("KNN\nn ="+str(knn_clf.n_neighbors)+"\n"+str(knn_accuracy))
axes[2].title.set_text("SVC\nC ="+str('{0:.2f}'.format(svc_clf.C))+"\n"+str(svc_accuracy))
axes[3].title.set_text("XGBoost\n"+str(model_accuracy))

In [ ]:
fig = plt.figure(figsize=(16,12))
ax = fig.add_axes([0,0,1,1])
falsePosRate, truePosRate, _ = roc_curve(testE,np.random.choice([0,1],size=len(testE)))
plot_roc_curve(knn5_clf,testD,testE,ax=ax,name="KNN, n = 5")
plot_roc_curve(knn_clf,testD,testE,ax=ax,name="KNN, n ="+str(knn_clf.n_neighbors))
plot_roc_curve(svc_clf,testD,testE,ax=ax,name="SVC, C ="+str('{0:.2f}'.format(svc_clf.C)))
plot_roc_curve(model,X_test_new,y_test,ax=ax,name="XGBoost")

ax.plot(falsePosRate, truePosRate, linestyle='--')

## <u><b> Conclusions </b></u>

Not only have we improved our model's accuracy, we also managed to do so using only 1 feature instead of 8 like with the old model, however, like explained prior we only reached an accuracy of 72.5%.

Although we managed to improve only by about 1% we managed to do so using 1/8 the amount of features so we could say this is a major improvement from the last model.